In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI模型花园 - BiomedCLIP

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_biomedclip.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_biomedclip.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"> 查看在GitHub上
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_biomedclip.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"> 在Vertex AI工作台中打开
    </a>（建议使用Python-3 CPU笔记本）
  </td>
</table>

## 概述

本笔记本演示了如何使用对比学习为[InBioMap](https://arxiv.org/pdf/2303.00915.pdf)提供服务，这是一个基于生物医学的视觉语言基础模型，其在PMC-15M上进行了预训练，该数据集包括从PubMed Central中提取的1500万个生物医学研究文章的图像标题对。InBioMap能够执行各种任务，包括零样本图像分类和视觉问答。

您可以在Hugging Face Hub上找到InBioMap的[模型检查点](https://huggingface.co/microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224)。

在本笔记本中，我们将向您展示如何：
- 使用Vertex AI为InBioMap提供服务
- 使用InBioMap进行零样本图像分类


### 成本

本教程使用Google Cloud的按需计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage定价](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

在您开始之前

**注意**：Jupyter会将前缀为`!`的行作为shell命令运行，并将前缀为`$`的Python变量内插到这些命令中。

只使用Colab
如果您正在使用Workbench，请运行以下命令并跳过此部分。

In [ ]:
import sys

if "google.colab" in sys.modules:
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

    # Restart the notebook kernel after installs.
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### 设置Google Cloud项目

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建账户时，会获得$300的免费信用额度，用于支付计算/存储成本。

1. [确保已为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

1. [创建一个Cloud Storage存储桶](https://cloud.google.com/storage/docs/creating-buckets)，用于存储实验输出。

1. [创建一个服务账号](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console)，并为其分配`Vertex AI用户`和`存储对象管理员`角色，以便将微调模型部署到Vertex AI终端节点。

填写以下变量以进行实验环境配置：

In [ ]:
# Cloud project id.
PROJECT_ID = ""  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = "us-central1"  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output.
BUCKET_URI = "gs://your_bucket_uri"  # @param {type:"string"}

! gcloud config set project $PROJECT_ID

# The service account looks like:
# '@.iam.gserviceaccount.com'
# Please go to https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console
# and create service account with `Vertex AI User` and `Storage Object Admin` roles.
# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = ""  # @param {type:"string"}

# The serving port.
SERVE_PORT = 7080

### 初始化 Vertex AI API

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

### 定义常量

In [ ]:
# The pre-built serving docker image.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-open-clip-serve"

### 定义效用函数

In [ ]:
from datetime import datetime
from typing import Tuple


def get_job_name_with_datetime(prefix: str) -> str:
    """Gets the job name with date time when triggering training or deployment
    jobs in Vertex AI.
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def download_image(url: str) -> str:
    """Downloads an image from the given URL.

    Args:
      url: The URL of the image to download.

    Returns:
      base64 encoded image.
    """
    !wget -O image.jpg $url
    !base64 image.jpg > image.txt
    return open("image.txt").read()


def deploy_model(
    model_name: str,
    model_id: str,
    service_account: str,
    task: str,
    precision: str,
    machine_type="n1-standard-8",
    accelerator_type="NVIDIA_TESLA_V100",
    accelerator_count=1,
) -> Tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Deploys trained models into Vertex AI."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    serving_env = {
        "MODEL": model_id,
        "TASK": task,
        "PRECISION": precision,
        "DEPLOY_SOURCE": "notebook",
    }
    # If the model_id is a GCS path, use artifact_uri to pass it to serving docker.
    artifact_uri = model_id if model_id.startswith("gs://") else None
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[SERVE_PORT],
        serving_container_predict_route="/predictions/transformers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
        artifact_uri=artifact_uri,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
    )
    return model, endpoint

### 使用图像进行推理
此部分将模型上传到模型注册表，并部署到端点上。

模型部署步骤将需要约10分钟完成。

In [ ]:
model_id = "hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224"

In [ ]:
model, endpoint = deploy_model(
    model_name=get_job_name_with_datetime(prefix="biomedclip-serve"),
    model_id=model_id,
    service_account=SERVICE_ACCOUNT,
    task="zero-shot-image-classification",
    precision="amp",
)
print("endpoint_name:", endpoint.name)

注意：模型权重将在部署成功后下载。因此，在上述模型部署步骤成功之后，需要额外等待5分钟的时间，然后再运行下面的步骤。否则，在向端点发送请求时，可能会出现`ServiceUnavailable: 503 502:Bad Gateway`错误。

一旦部署成功，您可以向端点发送文本和图像以进行零样本分类。

In [ ]:
import numpy as np

# # Loads an existing endpoint as below.
# endpoint_name = endpoint.name
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoint = aiplatform.Endpoint(aip_endpoint_name)

instances = [
    {
        "text": "This is a photo of adenocarcinoma histopathology",
        "image": download_image(
            "https://huggingface.co/microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224/resolve/main/example_data/biomed_image_classification_example_data/bone_X-ray.jpg"
        ),
    },
    {"text": "This is a photo of brain MRI"},
    {"text": "This is a photo of covid line chart"},
    {"text": "This is a photo of squamous cell carcinoma histopathology"},
    {"text": "This is a photo of immunohistochemistry histopathology"},
    {"text": "This is a photo of bone X-ray"},
    {"text": "This is a photo of chest X-ray"},
    {"text": "This is a photo of hematoxylin and eosin histopathology"},
    {"text": "This is a photo of pie chart"},
]
response = endpoint.predict(instances=instances)

print(response.predictions)

selected_idx = np.argmax(response.predictions[0])
print(f"Selected class: {instances[selected_idx]['text']}")

清理资源

In [ ]:
# Undeploy model and delete endpoint.
endpoint.delete(force=True)

# Delete models.
model.delete()